# Converting geotagged tweets to GEOJSON
(Reference: https://github.com/INFO-4602-5602/final-project-tweetsonamap/issues/4)

In [26]:
import pymongo, json, pprint, urllib.request, os.path, sys

Connect to the DB

    client = pymongo.MongoClient('mongodb://epic-analytics.cs.colorado.edu')
    db = client.matthew
    print(db.collection_names())
    tweets = db.tweets
    print(tweets.count(), "tweets")

In [65]:
client = pymongo.MongoClient('localhost')
tweets = client['matthew'].tweets

In [80]:
cursor = tweets.find({'geo' : {"$exists":True}}).limit(10000).batch_size(10)

In [84]:
idx = 0;
feats = []
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    if os.path.exists("/data/www/jennings/infovis/map_images/"+tweet_id+".jpg"):
        jsonObj = { "type" : "Feature",
                 "geometry" : {"type" : "Point", "coordinates" : list(reversed(t['geo']['coordinates']))},
                 "properties" : {  "id" : tweet_id,
                                   "user" : t['actor']['preferredUsername'],
                                   "text" : t['body'],
                                   "time" : 10, #Stubbed,
                                   "topic" : 1 #Stubbed
           }
         }
        feats.append(jsonObj)

KeyError: 'geo'

In [82]:
with open('/data/www/jennings/infovis-insta10000.geojson','w') as outFile:
    json.dump({'type':"FeatureCollection","features":feats},outFile)

## For Twitter

In [93]:
geo_image_tweets = {
    "verb":"post", # original tweet
    "twitter_extended_entities.media.0":{"$exists":True}, # at least 1 media entity
    "twitter_extended_entities.media": {"$all":[{"$elemMatch": { "type": "photo" }}]}, # all media entities are photos
    "location.geo.coordinates":{"$exists":True} # has geolocation (not necessarily a point)
}
cursor = tweets.find(geo_image_tweets).limit(10000).batch_size(10)

In [94]:
idx = 0;
feats = []
for t in cursor:
    tweet_id = t['id'].split(":")[2]
    if os.path.exists("/data/www/jennings/infovis/map_images/"+tweet_id+".jpg"):
        if t['location']['geo']['type'] == "Point":
            coords = t['location']['geo']['coordinates']
            jsonObj = { "type" : "Feature",
                     "geometry" : {"type" : "Point", "coordinates" : coords},
                     "properties" : {  "id" : tweet_id,
                                       "user" : t['actor']['preferredUsername'],
                                       "text" : t['body'],
                                       "time" : 10, #Stubbed,
                                       "topic" : 1 #Stubbed
               }
             }
            feats.append(jsonObj)

In [95]:
with open('/data/www/jennings/infovis-twitter10000.geojson','w') as outFile:
    json.dump({'type':"FeatureCollection","features":feats},outFile)